## Baseball Prediction: 1 - Data Wrangling
In this notebook we will wrangle data downloaded from www.retrosheet.org into a dataframe suitable for model building.  Specifically, for each game, we will calculate some team statistics over their past 162 games.

At the end, we save our dataframe to a file.  This file will be the starting point for the next notebook, in which we build our first model.

To use this notebook, you must first download the game logs here: https://www.retrosheet.org/gamelogs/index.html
Towards the bottom of the page there are links for ZIP files containing multiple seasons.  Download the 5 zip files for 1980-1989, 1990-1999, ... , 2020-2022, decompress them, and then move all of the single season files to a single directory.  You will then need the path to that directory for the variable `fname` below.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns',1000)
pd.set_option('display.max_rows',1000)

In [2]:
# Fill in the path to your data here...
fname = '/Users/brianlucena/Desktop/Work/baseball/data/game_data/'+'gl2022.txt'
df = pd.read_csv(fname, header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160
0,20220407,0,Thu,SDN,NL,1,ARI,NL,1,2,4,51,N,NaN,NaN,NaN,PHO01,35508.0,198,001100000,000000004,29,4,1,0,0,2,0,0,0,7,0,5,1,0,2,0,7,5,4,4,2,0,24,8,0,0,1,0,26,3,0,0,1,3,0,0,1,7,0,6,0,0,1,0,6,6,2,2,0,0,27,9,0,0,2,0,vanol901,Larry Vanover,belld901,Dan Bellino,barbs901,Sean Barber,valej901,Junior Valentine,NaN,(none),NaN,(none),melvb001,Bob Melvin,lovut001,Tony Lovullo,mantj002,Joe Mantiply,suarr002,Robert Suarez,NaN,(none),beers001,Seth Beer,darvy001,Yu Darvish,bumgm001,Madison Bumgarner,nolaa002,Austin Nola,2,machm001,Manny Machado,5,cronj001,Jake Cronenworth,4,voitl001,Luke Voit,10,myerw001,Wil Myers,9,hosme001,Eric Hosmer,3,profj001,Jurickson Profar,7,kim-h002,Ha-Seong Kim,6,grist001,Trent Grisham,8,varsd001,Daulton Varsho,8,martk001,Ketel Marte,4,perad001,David Peralta,7,walkc002,Christian Walker,3,smitp002,Pavin Smith,9,kellc002,Carson Kelly,2,beers001,Seth Beer,10,ellid002,Drew Ellis,5,perdg001,Gerardo Perdomo,6,NaN,Y
1,20220407,0,Thu,CIN,NL,1,ATL,NL,1,6,3,54,N,NaN,NaN,NaN,ATL03,40545.0,181,012003000,001000020,35,10,0,0,1,6,0,1,1,1,0,13,0,0,1,0,5,5,2,2,0,0,27,7,1,0,0,0,31,4,0,0,1,2,0,0,0,5,0,13,0,0,0,0,6,4,6,6,0,0,27,7,0,0,1,0,laynj901,Jerry Layne,wendh902,Hunter Wendelstedt,whitc901,Chad Whitson,hamaa901,Adam Hamari,NaN,(none),NaN,(none),belld002,David Bell,snitb801,Brian Snitker,mahlt001,Tyler Mahle,friem001,Max Fried,santt001,Tony Santillan,farmk001,Kyle Farmer,mahlt001,Tyler Mahle,friem001,Max Fried,indij001,Jonathan India,4,aquia001,Aristides Aquino,9,phamt001,Tommy Pham,7,vottj001,Joey Votto,3,stept001,Tyler Stephenson,2,senzn001,Nick Senzel,8,mousm001,Mike Moustakas,10,farmk001,Kyle Farmer,6,drurb001,Brandon Drury,5,rosae001,Eddie Rosario,9,olsom001,Matt Olson,3,rilea001,Austin Riley,5,ozunm001,Marcell Ozuna,7,albio001,Ozzie Albies,4,duvaa001,Adam Duvall,8,darnt001,Travis d'Arnaud,2,dicka001,Alex Dickerson,10,swand001,Dansby Swanson,6,NaN,Y
2,20220407,0,Thu,MIL,NL,1,CHN,NL,1,4,5,51,D,NaN,NaN,NaN,CHI11,35112.0,198,000100210,00003020x,33,10,4,0,0,4,0,2,1,4,0,9,1,0,2,0,9,3,5,5,0,0,24,10,0,0,1,0,29,8,3,0,1,5,0,1,1,4,0,7,0,1,1,0,6,6,4,4,0,0,27,9,0,0,2,0,barrt901,Ted Barrett,barkl901,Lance Barksdale,lentn901,Nic Lentz,cejan901,Nestor Ceja,NaN,(none),NaN,(none),counc001,Craig Counsell,rossd001,David Ross,givem001,Mychal Givens,ashba003,Aaron Ashby,robed002,David Robertson,happi001,Ian Happ,burnc002,Corbin Burnes,hendk001,Kyle Hendricks,wongk001,Kolten Wong,4,adamw002,Willy Adames,6,yelic001,Christian Yelich,7,mccua001,Andrew McCutchen,10,tellr001,Rowdy Tellez,3,renfh001,Hunter Renfroe,9,narvo001,Omar Narvaez,2,cainl001,Lorenzo Cain,8,petej002,Jace Peterson,5,orter001,Rafael Ortega,10,madrn001,Nick Madrigal,4,contw001,Willson Contreras,2,happi001,Ian Happ,7,schwf001,Frank Schwindel,3,suzus001,Seiya Suzuki,9,heywj001,Jason Heyward,8,wisdp001,Patrick Wisdom,5,hoern001,Nico Hoerner,6,NaN,Y
3,20220407,0,Thu,PIT,NL,1,SLN,NL,1,0,9,51,D,NaN,NaN,NaN,STL10,46256.0,188,000000000,13000104x,30,6,0,0,0,0,0,0,0,2,0,9,0,1,2,0,5,6,9,9,0,0,24,7,2,0,1,0,31,8,2,0,3,9,0,2,1,7,0,5,1,0,1,0,8,4,0,0,1,0,27,11,0,0,3,0,reynj901,Jim Reynolds,ticht901,Todd Tichenor,muchm901,Mike Muchlinski,merzd901,Dan Merzel,NaN,(none),NaN,(none),sheld801,Derek Shelton,marmo801,Oliver Marmol,waina001,Adam Wainwright,brubj001,JT Brubaker,NaN,(none),oneit001,Tyler O'Neill,brubj001,JT Brubaker,waina001,Adam Wainwright,voged001,Daniel Vogelbach,10,reynb001,Bryan Reynolds,8,hayek001,Ke'Bryan Hayes,5,tsuty001,Yoshi Tsutsugo,3,newmk001,Kevin Newman,6,tu

In [4]:
# Get colnames

In [5]:
colnames = ['date','dblheader_code','day_of_week','team_v','league_v','game_no_v',
           'team_h','league_h','game_no_h', 'runs_v', 'runs_h','outs_total','day_night',
            'completion_info','forfeit_info','protest_info','ballpark_id','attendance','game_minutes',
            'linescore_v','linescore_h',
           'AB_v','H_v','2B_v','3B_v','HR_v','RBI_v','SH_v','SF_v','HBP_v','BB_v','IBB_v','SO_v',
            'SB_v', 'CS_v','GIDP_v','CI_v','LOB_v',
            'P_num_v','ERind_v','ERteam_v','WP_v','balk_v',
            'PO_v','ASST_v','ERR_v','PB_v','DP_v','TP_v',
           'AB_h', 'H_h', '2B_h', '3B_h', 'HR_h', 'RBI_h', 'SH_h', 'SF_h', 'HBP_h', 'BB_h', 'IBB_h','SO_h',
            'SB_h', 'CS_h', 'GIDP_h', 'CI_h', 'LOB_h',
            'P_num_h', 'ERind_h', 'ERteam_h', 'WP_h', 'balk_h',
            'PO_h', 'ASST_h', 'ERR_h', 'PB_h', 'DP_h', 'TP_h',
            'ump_HB_id', 'ump_HB_name','ump_1B_id', 'ump_1B_name','ump_2B_id', 'ump_2B_name',
            'ump_3B_id', 'ump_3B_name','ump_LF_id', 'ump_LF_name','ump_RF_id', 'ump_RF_name',
            'mgr_id_v', 'mgr_name_v', 'mgr_id_h', 'mgr_name_h',
            'pitcher_id_w','pitcher_name_w','pitcher_id_l','pitcher_name_l','pitcher_id_s','pitcher_name_s',
            'GWRBI_id','GWRBI_name','pitcher_start_id_v','pitcher_start_name_v','pitcher_start_id_h','pitcher_start_name_h',
            'batter1_name_v', 'batter1_id_v', 'batter1_pos_v', 'batter2_name_v', 'batter2_id_v', 'batter2_pos_v',
            'batter3_name_v', 'batter3_id_v', 'batter3_pos_v', 'batter4_name_v', 'batter4_id_v', 'batter4_pos_v',
            'batter5_name_v', 'batter5_id_v', 'batter5_pos_v', 'batter6_name_v', 'batter6_id_v', 'batter6_pos_v',
            'batter7_name_v', 'batter7_id_v', 'batter7_pos_v', 'batter8_name_v', 'batter8_id_v', 'batter8_pos_v',
            'batter9_name_v', 'batter9_id_v', 'batter9_pos_v', 'batter1_name_h', 'batter1_id_h', 'batter1_pos_h',
            'batter2_name_h', 'batter2_id_h', 'batter2_pos_h', 'batter3_name_h', 'batter3_id_h', 'batter3_pos_h',
            'batter4_name_h', 'batter4_id_h', 'batter4_pos_h', 'batter5_name_h', 'batter5_id_h', 'batter5_pos_h',
            'batter6_name_h', 'batter6_id_h', 'batter6_pos_h', 'batter7_name_h', 'batter7_id_h', 'batter7_pos_h',
            'batter8_name_h', 'batter8_id_h', 'batter8_pos_h', 'batter9_name_h', 'batter9_id_h', 'batter9_pos_h',           
           'misc_info','acqui_info'
           ]


In [6]:
df.columns = colnames

In [7]:
df.sample(10)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info
1619,20220807,0,Sun,ATL,NL,110,NYN,NL,109,2,5,51,D,NaN,NaN,NaN,NYC20,37717.0,178,000002000,00401000x,29,2,0,0,1,2,0,0,0,1,0,19,0,0,0,0,1,5,5,5,1,0,24,7,0,0,0,0,34,10,4,0,0,4,0,0,0,3,1,8,0,0,0,0,8,3,2,2,0,0,27,2,0,0,0,0,buckc901,CB Bucknor,sches901,Stu Scheurwater,nelsj901,Jeff Nelson,gonzm901,Manny Gonzalez,NaN,(none),NaN,(none),snitb801,Brian Snitker,showb801,Buck Showalter,degrj001,Jacob deGrom,stris002,Spencer Strider,diaze006,Edwin Diaz,alonp001,Pete Alonso,stris002,Spencer Strider,degrj001,Jacob deGrom,swand001,Dansby Swanson,6,olsom001,Matt Olson,3,rilea001,Austin Riley,5,rosae001,Eddie Rosario,7,contw002,William Contreras,2,grosr001,Robbie Grossman,9,ozunm001,Marcell Ozuna,10,harrm004,Michael Harris,8,adrie001,Ehire Adrianza,4,nimmb001,Brandon Nimmo,8,marts002,Starling Marte,9,lindf001,Francisco Lindor,6,alonp001,Pete Alonso,3,voged001,Daniel Vogelbach,10,mcnej002,Jeff McNeil,4,canhm001,Mark Canha,7,guill001,Luis Guillorme,5,nidot001,Tomas Nido,2,NaN,Y
1911,20220828,0,Sun,DET,AL,128,TEX,AL,127,9,8,54,D,NaN,NaN,NaN,ARL03,24938.0,215,031203000,000002033,39,13,5,0,1,9,0,0,1,4,0,8,0,0,1,0,8,5,8,8,3,0,27,15,3,1,3,0,37,10,2,0,4,8,0,0,0,4,0,6,1,0,3,0,6,6,9,9,0,0,27,9,0,0,1,0,ticht901,Todd Tichenor,bacoj901,John Bacon,littw901,Will Little,drakr901,Rob Drake,NaN,(none),NaN,(none),hinca001,A.J. Hinch,beast801,Tony Beasley,hutcd001,Drew Hutchison,arihk001,Kohei Arihara,jimej003,Joe Jimenez,carpk001,Kerry Carpenter,hutcd001,Drew Hutchison,arihk001,Kohei Arihara,greer003,Riley Greene,8,reyev001,Victor Reyes,9,baezj001,Javier Baez,6,casth001,Harold Castro,3,haase001,Eric Haase,2,carpk001,Kerry Carpenter,10,candj002,Jeimer Candelario,5,clemk001,Kody Clemens,4,badda001,Akil Baddoo,7,semim001,Marcus Semien,4,seagc001,Corey Seager,6,lowen001,Nate Lowe,3,garca005,Adolis Garcia,9,calhk001,Kole Calhoun,7,tavel001,Leody Taveras,8,millb002,Brad Miller,10,vilom001,Meibrys Viloria,2,durae002,Ezequiel Duran,5,NaN,Y
464,20220512,0,Thu,PHI,NL,32,LAN,NL,30,9,7,54,N,NaN,NaN,NaN,LOS03,46539.0,209,130111002,001002040,37,12,2,2,2,8,0,2,0,1,0,10,2,0,0,0,4,6,7,7,0,0,27,5,0,0,0,0,39,12,3,0,1,7,0,0,0,5,0,10,0,0,0,0,10,3,9,9,1,0,27,5,1,0,0,0,hobep901,Pat Hoberg,emmep901,Paul Emmel,johna901,Adrian Johnson,mosce901,Edwin Moscoso,NaN,(none),NaN,(none),giraj001,Joe Girardi,robed001,Dave Roberts,bella001,Andrew Bellatti

## Create a Table with every game since 1980

In [8]:
df = pd.DataFrame()
for year in range(1980,2023):
    fname = '/Users/brianlucena/Desktop/Work/baseball/data/game_data/gl' +str(year)+'.txt'
    df_temp = pd.read_csv(fname, header=None)
    df_temp.columns = colnames
    df_temp['season'] = year
    df = pd.concat((df, df_temp))


In [9]:
df.shape

(96276, 162)

In [10]:
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96276 entries, 0 to 2429
Data columns (total 162 columns):
 #    Column                Non-Null Count  Dtype  
---   ------                --------------  -----  
 0    date                  96276 non-null  int64  
 1    dblheader_code        96276 non-null  int64  
 2    day_of_week           96276 non-null  object 
 3    team_v                96276 non-null  object 
 4    league_v              96276 non-null  object 
 5    game_no_v             96276 non-null  int64  
 6    team_h                96276 non-null  object 
 7    league_h              96276 non-null  object 
 8    game_no_h             96276 non-null  int64  
 9    runs_v                96276 non-null  int64  
 10   runs_h                96276 non-null  int64  
 11   outs_total            96276 non-null  int64  
 12   day_night             96276 non-null  object 
 13   completion_info       83 non-null     object 
 14   forfeit_info          1 non-null      object 
 15   p

In [11]:
# Create columns for outcomes

In [12]:
## Calculate a few useful columns
df['run_diff'] = df['runs_h']-df['runs_v']
df['home_victory'] = (df['run_diff']>0).astype(int)
df['run_total'] = df['runs_h'].copy()+df['runs_v'].copy()
df['date_dblhead'] = (df['date'].astype(str) + df['dblheader_code'].astype(str)).astype(int)


In [13]:
# Do some basic exploration

In [14]:
df.home_victory.mean()

0.538368856205077

### Big picture - have outcome, need features
- Would like to start simple, base features on team results over past *n* games
- Need to do a bit of "data wrangling"
- First, we need to have each team's games in chronological order
- Then can use pandas 'rolling' functionality to aggregate over windows in the past
- Immediate goal, get batting average, on-base percentage, and slugging percentage
- Need "running values" based on the past, not including the current game

In [15]:
# Let's look at a single team's games

In [16]:
df_mets = df.loc[((df.team_v=='NYN') | (df.team_h=='NYN')), :]

In [17]:
df_mets.shape

(6736, 166)

In [18]:
df_mets.head(100)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead
8,19800410,0,Thu,CHN,NL,1,NYN,NL,1,2,5,51,D,NaN,NaN,NaN,NYC17,12219.0,143,000001100,01000400x,34,7,3,0,0,2,0,0,0,1,0,3,0,0,1,0,6,3,4,4,0,0,24,12,1,0,1,0,29,8,1,0,0,4,0,1,0,6,2,5,0,2,1,0,7,2,2,2,0,0,27,9,1,0,1,0,kiblj901,John Kibler,froeb901,Bruce Froemming,tatat901,Terry Tata,rennd901,Dutch Rennert,NaN,(none),NaN,(none),gomep101,Preston Gomez,torrj101,Joe Torre,swanc001,Craig Swan,reusr001,Rick Reuschel,allen001,Neil Allen,NaN,(none),reusr001,Rick Reuschel,swanc001,Craig Swan,randl101,Lenny Randle,4,dejei001,Ivan De Jesus,6,buckb001,Bill Buckner,3,kingd001,Dave Kingman,7,hendk101,Ken Henderson,9,ontis101,Steve Ontiveros,5,lezcc101,Carlos Lezcano,8,blact101,Tim Blackwell,2,reusr001,Rick Reuschel,1,tavef101,Frank Taveras,6,madde101,Elliott Maddox,5,mazzl001,Lee Mazzilli,3,hends001,Steve Henderson,7,jorgm001,Mike Jorgensen,9,steaj001,John Stearns,2,moraj101,Jerry Morales,8,flynd001,Doug Flynn,4,swanc001,Craig Swan,1,NaN,Y,1980,3,1,7,198004100
19,19800411,0,Fri,CHN,NL,2,NYN,NL,2,7,5,54,D,NaN,NaN,NaN,NYC17,4460.0,168,023100100,012000101,37,11,1,0,5,7,0,0,0,7,1,5,1,0,1,0,10,3,5,5,1,0,27,17,0,0,1,0,38,13,3,0,0,5,1,0,0,3,0,8,1,0,1,0,10,5,7,7,0,0,27,12,0,0,1,0,froeb901,Bruce Froemming,tatat901,Terry Tata,rennd901,Dutch Rennert,brocf901,Fred Brocklander,NaN,(none),NaN,(none),gomep101,Preston Gomez,torrj101,Joe Torre,lampd001,Dennis Lamp,burrr001,Ray Burris,suttb001,Bruce Sutter,kingd001,Dave Kingman,lampd001,Dennis Lamp,burrr001,Ray Burris,randl101,Lenny Randle,4,dejei001,Ivan De Jesus,6,buckb001,Bill Buckner,3,kingd001,Dave Kingman,7,hendk101,Ken Henderson,9,martj001,Jerry Martin,8,ontis101,Steve Ontiveros,5,footb101,Barry Foote,2,lampd001,Dennis Lamp,1,tavef101,Frank Taveras,6,madde101,Elliott Maddox,5,mazzl001,Lee Mazzilli,3,hends001,Steve Henderson,7,jorgm001,Mike Jorgensen,9,steaj001,John Stearns,2,moraj101,Jerry Morales,8,flynd001,Doug Flynn,4,burrr001,Ray Burris,1,NaN,Y,1980,-2,0,12,198004110
31,19800412,0,Sat,CHN,NL,3,NYN,NL,3,6,3,54,D,NaN,NaN,NaN,NYC17,10781.0,149,000000420,010001100,37,14,1,0,2,6,0,1,0,3,1,2,1,1,3,0,8,2,2,2,0,0,27,11,2,0,1,0,29,6,0,0,2,3,0,1,1,5,1,6,1,1,0,0,6,4,5,5,2,0,27,16,2,0,4,0,tatat901,Terry Tata,rennd901,Dutch Rennert,brocf901,Fred Brocklander,kiblj901,John Kibler,NaN,(none),NaN,(none),gomep101,Preston Gomez,t

In [19]:
# Write a function to create a team-specific data frame, given the team

In [20]:
def strip_suffix(x, suff):
    if x.endswith(suff):
        return(x[:-len(suff)])
    else:
        return(x)

visit_cols = [col for col in df.columns if not col.endswith('_h')]
visit_cols_stripped = [strip_suffix(col,'_v') for col in visit_cols]
home_cols = [col for col in df.columns if not col.endswith('_v')]
home_cols_stripped = [strip_suffix(col,'_h') for col in home_cols]

## This subsets the game level df by team, to aggregate team statistics easily
## We also create rolling sums with an offset, so that the rollsum number represents
## statistics up to, but not including, the game in question

def create_team_df(team):
    df_team_v = df[(df.team_v==team)]
    opponent = df_team_v['team_h']
    df_team_v = df_team_v[visit_cols]
    df_team_v.columns = visit_cols_stripped
    df_team_v['home_game'] = 0
    df_team_v['opponent'] = opponent

    df_team_h = df[(df.team_h==team)]
    opponent = df_team_h['team_v']
    df_team_h = df_team_h[home_cols]
    df_team_h.columns = home_cols_stripped
    df_team_h['home_game'] = 1
    df_team_h['opponent'] = opponent


    df_team = pd.concat((df_team_h, df_team_v))
    df_team.sort_values(['date', 'game_no'],inplace=True)
    
    for winsize in [162,30]:
        suff = str(winsize)
        for raw_col in ['AB','H','2B','3B','HR','BB','runs','SB','CS','ERR']:
            new_col = 'rollsum_'+raw_col+'_'+suff
            df_team[new_col] = df_team[raw_col].rolling(winsize, closed='left').sum()

        df_team['rollsum_BATAVG_'+suff] = df_team['rollsum_H_'+suff] / df_team['rollsum_AB_'+suff]
        df_team['rollsum_OBP_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_BB_'+suff]) / (
                                    df_team['rollsum_AB_'+suff]+df_team['rollsum_BB_'+suff])
        df_team['rollsum_SLG_'+suff] = (df_team['rollsum_H_'+suff] + df_team['rollsum_2B_'+suff] 
                                 + 2*df_team['rollsum_3B_'+suff]+ 
                                3*df_team['rollsum_HR_'+suff] ) / (df_team['rollsum_AB_'+suff])
        df_team['rollsum_OBS_'+suff] = df_team['rollsum_OBP_'+suff] + df_team['rollsum_SLG_'+suff]
    
    df_team['season_game'] = df_team['season']*1000 + df_team['game_no']
    df_team.set_index('season_game', inplace=True)
    return(df_team)

In [21]:
df_mets = create_team_df('NYN')

In [22]:
df_mets.sample(10)

,date,dblheader_code,day_of_week,team,league,game_no,runs,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore,AB,H,2B,3B,HR,RBI,SH,SF,HBP,BB,IBB,SO,SB,CS,GIDP,CI,LOB,P_num,ERind,ERteam,WP,balk,PO,ASST,ERR,PB,DP,TP,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id,mgr_name,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id,pitcher_start_name,batter1_name,batter1_id,batter1_pos,batter2_name,batter2_id,batter2_pos,batter3_name,batter3_id,batter3_pos,batter4_name,batter4_id,batter4_pos,batter5_name,batter5_id,batter5_pos,batter6_name,batter6_id,batter6_pos,batter7_name,batter7_id,batter7_pos,batter8_name,batter8_id,batter8_pos,batter9_name,batter9_id,batter9_pos,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,home_game,opponent,rollsum_AB_162,rollsum_H_162,rollsum_2B_162,rollsum_3B_162,rollsum_HR_162,rollsum_BB_162,rollsum_runs_162,rollsum_SB_162,rollsum_CS_162,rollsum_ERR_162,rollsum_BATAVG_162,rollsum_OBP_162,rollsum_SLG_162,rollsum_OBS_162,rollsum_AB_30,rollsum_H_30,rollsum_2B_30,rollsum_3B_30,rollsum_HR_30,rollsum_BB_30,rollsum_runs_30,rollsum_SB_30,rollsum_CS_30,rollsum_ERR_30,rollsum_BATAVG_30,rollsum_OBP_30,rollsum_SLG_30,rollsum_OBS_30
season_game,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1995130,19950916,0,Sat,NYN,NL,130,10,51,D,NaN,NaN,NaN,NYC17,18351.0,187,05103001x,32,11,3,0,1,8,0,0,0,10,0,9,1,1,1,0,8,5,7,7,0,1,27,6,1,1,0,0,poncl901,Larry Poncino,ripps901,Steve Rippley,quicj901,Jim Quick,davib902,Bob Davidson,NaN,(none),NaN,(none),greed101,Dallas Green,telgd001,Dave Telgheder,greet001,Tommy Greene,franj001,John Franco,telgd001,Dave Telgheder,telgd001,Dave Telgheder,bufod001,Damon Buford,7,vizcj001,Jose Vizcaino,6,everc001,Carl Everett,9,kentj001,Jeff Kent,4,brogr001,Rico Brogna,3,hundt001,Todd Hundley,2,thomr004,Ryan Thompson,8,bogat001,Tim Bogar,5,telgd001,Dave Telgheder,1,NaN,Y,1995,2,1,18,199509160,1,PHI,5639.0,1483.0,243.0,37.0,147.0,478.0,721.0,57.0,46.0,129.0,0.262990,0.320582,0.397411,0.717993,990.0,242.0,34.0,3.0,27.0,94.0,133.0,5.0,11.0,24.0,0.244444,0.309963,0.366667,0.676630
1990154,19900925,0,Tue,NYN,NL,154,3,54,N,NaN,NaN,NaN,MON02,11959.0,188,001000200,36,9,0,1,0,2,0,0,0,3,0,11,1,0,0,0,9,3,1,1,0,0,27,5,1,0,0,0,demud901,Dana DeMuth,grege901,Eric Gregg,crawj901,Jerry Crawford,harvd901,Doug Harvey,NaN,(none),NaN,(none),harrb101,Bud Harrelson,penaa001,Alejandro Pena,nabhc001,Chris Nabholz,NaN,(none),magad001,Dave Magadan,ferns001,Sid Fernandez,millk001,Keith Miller,8,herrt001,Tom Herr,4,teuft001,Tim Teufel,3,mcrek001,Kevin McReynolds,7,tablp001,Pat Tabler,9,jeffg001,Gregg Jefferies,5,johnh001,Howard Johnson,6,obric001,Charlie O'Brien,2,ferns001,Sid Fernandez,1,NaN,Y,1990,-2,0,4,199009250,0,MON,5518.0,1416.0,277.0,20.0,175.0,536.0,786.0,113.0,34.0,138.0,0.256615,0.322431,0.409206,0.731638,982.0,228.0,46.0,2.0,29.0,101.0,134.0,13.0,5.0,21.0,0.232179,0.303786,0.371690,0.675476
1989002,19890405,0,Wed,NYN,NL,2,1,54,D,NaN,NaN,NaN,NYC17,17873.0,149,010000000,31,5,2,0,0,1,0,0,0,2,0,3,0,0,0,0,5,3,2,2,0,0,27,14,3,0,1,0,monte901,Ed Montague,marsr901,Randy Marsh,darlg901,Gary Darling,wendh901,Harry Wendelstedt,NaN,(none),NaN,(none),johnd105,Davey Johnson,delej001,Jose DeLeon,ojedb001,Bob Ojeda,worrt001,Todd Worrell,guerp001,Pedro Guerrero,ojedb001,Bob Ojeda,dyksl001,Lenny Dykstra,8,jeffg001,Gregg Jefferies,4,hernk001,Keith Hernandez,3,strad001,Darryl Strawberry,9,mcrek001,Kevin McReynolds,7,cartg001,Gary Carter,2,johnh001,Howard Johnson,5,elstk001,Kevin Elster,6,ojedb001,Bob Ojeda,1,NaN,Y,1989,-2,0,4,198904050,1,SLN,5474.0,1411.0,257.0,25.0,155.0,552.0,722.0,144.0,55.0,116.0,0.257764,0.325755,0.398794,0.724549,1005.0,279.0,61.0,4.0,39.0,103.0,163.0,27.0,14.0,24.0,0.277612,0.344765,0.462687,0.80745

In [23]:
# Make a dictionary that maps a team name to it's data frame

In [24]:
# Create the team level dataframe for each team - put in dict for easy access
team_data_dict = {}
for team in df.team_v.unique():
    team_data_dict[team] = create_team_df(team)

In [25]:
# Go through the rows of the main dataframe, and augment it with home and visiting teams' features

In [26]:
## Create a variety of summarized statistics for each game
## For each game, we look up the home and visiting team in the team
## data dictionary, and then look up the game, and pull the relevant stats

BATAVG_162_h = np.zeros(df.shape[0])
BATAVG_162_v = np.zeros(df.shape[0])
OBP_162_h = np.zeros(df.shape[0])
OBP_162_v = np.zeros(df.shape[0])
SLG_162_h = np.zeros(df.shape[0])
SLG_162_v = np.zeros(df.shape[0])
OBS_162_h = np.zeros(df.shape[0])
OBS_162_v = np.zeros(df.shape[0])
SB_162_h = np.zeros(df.shape[0])
SB_162_v = np.zeros(df.shape[0])
CS_162_h = np.zeros(df.shape[0])
CS_162_v = np.zeros(df.shape[0])
ERR_162_h = np.zeros(df.shape[0])
ERR_162_v = np.zeros(df.shape[0])
BATAVG_30_h = np.zeros(df.shape[0])
BATAVG_30_v = np.zeros(df.shape[0])
OBP_30_h = np.zeros(df.shape[0])
OBP_30_v = np.zeros(df.shape[0])
SLG_30_h = np.zeros(df.shape[0])
SLG_30_v = np.zeros(df.shape[0])
OBS_30_h = np.zeros(df.shape[0])
OBS_30_v = np.zeros(df.shape[0])
SB_30_h = np.zeros(df.shape[0])
SB_30_v = np.zeros(df.shape[0])
CS_30_h = np.zeros(df.shape[0])
CS_30_v = np.zeros(df.shape[0])
ERR_30_h = np.zeros(df.shape[0])
ERR_30_v = np.zeros(df.shape[0])
i=0
for index, row in df.iterrows():
    if i%1000==0:
        print(i)
    home_team = row['team_h']
    visit_team = row['team_v']
    game_index_v = row['season']*1000 + row['game_no_v']
    game_index_h = row['season']*1000 + row['game_no_h']
    BATAVG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_162']
    BATAVG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_162']
    OBP_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_162']
    OBP_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_162']
    SLG_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_162']
    SLG_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_162']
    OBS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_162']
    OBS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_162']
    SB_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_162']
    SB_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_162']
    CS_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_162']
    CS_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_162']
    ERR_162_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_162']
    ERR_162_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_162']
    BATAVG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_BATAVG_30']
    BATAVG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_BATAVG_30']
    OBP_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBP_30']
    OBP_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBP_30']
    SLG_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SLG_30']
    SLG_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SLG_30']
    OBS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_OBS_30']
    OBS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_OBS_30']
    SB_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_SB_30']
    SB_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_SB_30']
    CS_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_CS_30']
    CS_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_CS_30']
    ERR_30_h[i] = team_data_dict[home_team].loc[game_index_h,'rollsum_ERR_30']
    ERR_30_v[i] = team_data_dict[visit_team].loc[game_index_v,'rollsum_ERR_30']
    i+=1
    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000


In [27]:
## We then put the constructed arrays into the main game level dataframe
df['BATAVG_162_h'] = BATAVG_162_h
df['BATAVG_162_v'] = BATAVG_162_v
df['OBP_162_h'] = OBP_162_h
df['OBP_162_v'] = OBP_162_v
df['SLG_162_h'] = SLG_162_h
df['SLG_162_v'] = SLG_162_v
df['OBS_162_h'] = OBS_162_h
df['OBS_162_v'] = OBS_162_v
df['SB_162_h'] = SB_162_h
df['SB_162_v'] = SB_162_v
df['CS_162_h'] = CS_162_h
df['CS_162_v'] = CS_162_v
df['ERR_162_h'] = ERR_162_h
df['ERR_162_v'] = ERR_162_v
df['BATAVG_30_h'] = BATAVG_30_h
df['BATAVG_30_v'] = BATAVG_30_v
df['OBP_30_h'] = OBP_30_h
df['OBP_30_v'] = OBP_30_v
df['SLG_30_h'] = SLG_30_h
df['SLG_30_v'] = SLG_30_v
df['OBS_30_h'] = OBS_30_h
df['OBS_30_v'] = OBS_30_v
df['SB_30_h'] = SB_30_h
df['SB_30_v'] = SB_30_v
df['CS_30_h'] = CS_30_h
df['CS_30_v'] = CS_30_v
df['ERR_30_h'] = ERR_30_h
df['ERR_30_v'] = ERR_30_v


In [28]:
df.shape

(96276, 194)

In [29]:
df.sample(5)

,date,dblheader_code,day_of_week,team_v,league_v,game_no_v,team_h,league_h,game_no_h,runs_v,runs_h,outs_total,day_night,completion_info,forfeit_info,protest_info,ballpark_id,attendance,game_minutes,linescore_v,linescore_h,AB_v,H_v,2B_v,3B_v,HR_v,RBI_v,SH_v,SF_v,HBP_v,BB_v,IBB_v,SO_v,SB_v,CS_v,GIDP_v,CI_v,LOB_v,P_num_v,ERind_v,ERteam_v,WP_v,balk_v,PO_v,ASST_v,ERR_v,PB_v,DP_v,TP_v,AB_h,H_h,2B_h,3B_h,HR_h,RBI_h,SH_h,SF_h,HBP_h,BB_h,IBB_h,SO_h,SB_h,CS_h,GIDP_h,CI_h,LOB_h,P_num_h,ERind_h,ERteam_h,WP_h,balk_h,PO_h,ASST_h,ERR_h,PB_h,DP_h,TP_h,ump_HB_id,ump_HB_name,ump_1B_id,ump_1B_name,ump_2B_id,ump_2B_name,ump_3B_id,ump_3B_name,ump_LF_id,ump_LF_name,ump_RF_id,ump_RF_name,mgr_id_v,mgr_name_v,mgr_id_h,mgr_name_h,pitcher_id_w,pitcher_name_w,pitcher_id_l,pitcher_name_l,pitcher_id_s,pitcher_name_s,GWRBI_id,GWRBI_name,pitcher_start_id_v,pitcher_start_name_v,pitcher_start_id_h,pitcher_start_name_h,batter1_name_v,batter1_id_v,batter1_pos_v,batter2_name_v,batter2_id_v,batter2_pos_v,batter3_name_v,batter3_id_v,batter3_pos_v,batter4_name_v,batter4_id_v,batter4_pos_v,batter5_name_v,batter5_id_v,batter5_pos_v,batter6_name_v,batter6_id_v,batter6_pos_v,batter7_name_v,batter7_id_v,batter7_pos_v,batter8_name_v,batter8_id_v,batter8_pos_v,batter9_name_v,batter9_id_v,batter9_pos_v,batter1_name_h,batter1_id_h,batter1_pos_h,batter2_name_h,batter2_id_h,batter2_pos_h,batter3_name_h,batter3_id_h,batter3_pos_h,batter4_name_h,batter4_id_h,batter4_pos_h,batter5_name_h,batter5_id_h,batter5_pos_h,batter6_name_h,batter6_id_h,batter6_pos_h,batter7_name_h,batter7_id_h,batter7_pos_h,batter8_name_h,batter8_id_h,batter8_pos_h,batter9_name_h,batter9_id_h,batter9_pos_h,misc_info,acqui_info,season,run_diff,home_victory,run_total,date_dblhead,BATAVG_162_h,BATAVG_162_v,OBP_162_h,OBP_162_v,SLG_162_h,SLG_162_v,OBS_162_h,OBS_162_v,SB_162_h,SB_162_v,CS_162_h,CS_162_v,ERR_162_h,ERR_162_v,BATAVG_30_h,BATAVG_30_v,OBP_30_h,OBP_30_v,SLG_30_h,SLG_30_v,OBS_30_h,OBS_30_v,SB_30_h,SB_30_v,CS_30_h,CS_30_v,ERR_30_h,ERR_30_v
800,20130531,0,Fri,SEA,AL,55,MIN,AL,52,3,0,54,N,NaN,NaN,NaN,MIN04,31430.0,173,000003000,000000000,32,8,0,0,1,3,1,0,1,4,0,5,1,0,3,0,8,3,0,0,0,0,27,14,0,0,1,0,33,7,2,0,0,0,0,0,0,2,0,6,0,0,1,0,8,3,3,3,0,0,27,12,1,0,3,0,barrl901,Lance Barrett,mcclt901,Tim McClelland,hudsm901,Marvin Hudson,bellw901,Wally Bell,NaN,(none),NaN,(none),wedge001,Eric Wedge,gardr001,Ron Gardenhire,iwakh001,Hisashi Iwakuma,pelfm001,Mike Pelfrey,wilht001,Tom Wilhelmsen,morak001,Kendrys Morales,iwakh001,Hisashi Iwakuma,pelfm001,Mike Pelfrey,chave002,Endy Chavez,9,bay-j001,Jason Bay,7,seagk001,Kyle Seager,5,morak001,Kendrys Morales,3,ibanr001,Raul Ibanez,10,saunm001,Michael Saunders,8,frann001,Nick Franklin,4,sucrj001,Jesus Sucre,2,ryanb002,Brendan Ryan,6,carrj001,Jamey Carroll,5,dozib001,Brian Dozier,4,mauej001,Joe Mauer,2,willj004,Josh Willingham,7,mornj001,Justin Morneau,3,doumr001,Ryan Doumit,10,parmc001,Chris Parmelee,9,hicka001,Aaron Hicks,8,florp001,Pedro Florimon,6,NaN,Y,2013,-3,0,3,201305310,0.260110,0.234846,0.320739,0.295808,0.392125,0.374410,0.712865,0.670218,128.0,85.0,37.0,31.0,98.0,70.0,0.241706,0.240506,0.306759,0.310954,0.390521,0.407011,0.697280,0.717965,13.0,11.0,5.0,4.0,7.0,12.0
1828,20020817,0,Sat,MIL,NL,122,PIT,NL,123,0,5,51,N,NaN,NaN,NaN,PIT08,25277.0,161,000000000,00000500x,31,6,1,0,0,0,0,0,0,1,1,5,2,1,1,0,5,3,5,5,0,0,24,8,0,0,0,0,28,6,2,0,0,5,1,0,2,6,1,4,0,1,0,0,8,4,0,0,0,0,27,12,1,0,1,0,nelsj901,Jeff Nelson,kulpr901,Ron Kulpa,joycj901,Jim Joyce,scotd901,Dale Scott,NaN,(none),NaN,(none),roysj001,Jerry Royster,mccll001,Lloyd McClendon,meadb001,Brian Meadows,cabrj001,Jose Cabrera,NaN,(none),hyzda001,Adam Hyzdu,ostij001,Jimmy Osting,meadb001,Brian Meadows,sanca003,Alex Sanchez,8,youne001,Eric Young,4,hammj001,Jeffrey Hammonds,7,sexsr001,Richie Sexson,3,staim001,Matt Stairs,9,hernj001,Jose Hernandez,6,lorem001,Mark Loretta,5,fabrj001,Jorge Fabregas,2,ostij001,Jimmy Osting,1,reesp001,Pokey Reese,4,wilsj002,Jack Wilson,6,kendj001,Jason Kendall,2,ramia001,Aramis Ramirez,5,younk001,Ke

In [30]:
df.to_csv('df_bp1.csv', index=False)
